In [1]:
from typing import Union
import os
from os.path import join
import requests
import earthaccess
import pandas as pd
import datetime as dt
from datetime import date
from dateutil import parser
import geopandas
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon, box
from shapely.geometry.polygon import orient
from sentinel_tiles import sentinel_tiles
from emit_tools.generate_CMR_daterange_string import generate_CMR_daterange_string
from emit_tools.search_EMIT_L2A_reflectance import search_EMIT_L2A_reflectance
from emit_tools.download_file import download_file
from emit_tools.download_EMIT_L2A_scene import download_EMIT_L2A_scene
from emit_tools.find_concept_ID import find_concept_ID
from emit_tools.CMR_collection_search_short_name_URL import CMR_collection_search_short_name_URL
from emit_tools.CMR_collection_search_DOI_URL import CMR_collection_search_DOI_URL
from emit_tools.concept_ID_from_short_name import concept_ID_from_short_name
from emit_tools.concept_ID_from_DOI import concept_ID_from_DOI
from emit_tools.constants import *

In [2]:
geojson_filename = "11SPS.geojson"
start_date = "2023-06-01"
end_date = "2024-05-30"

In [3]:
polygon = sentinel_tiles.footprint("11SPS")
polygon.wkt

'POLYGON ((-115.92423373599996 33.43490918800006, -114.74363157999994 33.419046707000064, -114.76862412 32.429359161000036, -115.93615876899997 32.444635394000045, -115.92423373599996 33.43490918800006))'

In [4]:
reflectance_granules_df = search_EMIT_L2A_reflectance(
    start_date=start_date, 
    end_date=end_date, 
    target_area=polygon
)

reflectance_granules_df

,date_UTC,product,filename,URL,cloud_percent,level,datetime_UTC,orbit,scene,collection,type,geometry
0,2023-06-02,L2A_RFL,EMIT_L2A_RFL_001_20230602T193753_2315313_003.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,17,L2A,2023-06-02 19:37:53,2315313,3,1,data,"MULTIPOLYGON (((-115.69995 33.0349, -116.4238 ..."
1,2023-06-02,L2A_RFLUNCERT,EMIT_L2A_RFLUNCERT_001_20230602T193753_2315313...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,17,L2A,2023-06-02 19:37:53,2315313,3,1,data,"MULTIPOLYGON (((-115.69995 33.0349, -116.4238 ..."
2,2023-06-02,L2A_MASK,EMIT_L2A_MASK_001_20230602T193753_2315313_003.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,17,L2A,2023-06-02 19:37:53,2315313,3,1,data,"MULTIPOLYGON (((-115.69995 33.0349, -116.4238 ..."
3,2023-06-02,L2A_RFL,EMIT_L2A_RFL_001_20230602T193753_2315313_003.png,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,17,L2A,2023-06-02 19:37:53,2315313,3,1,browse,"MULTIPOLYGON (((-115.69995 33.0349, -116.4238 ..."
4,2023-06-02,L2A_RFL,EMIT_L2A_RFL_001_20230602T193753_2315313_003.n...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,17,L2A,2023-06-02 19:37:53,2315313,3,1,unknown,"MULTIPOLYGON (((-115.69995 33.0349, -116.4238 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2,2024-05-21,L2A_RFL,EMIT_L2A_RFL_001_20240521T232740_2414216_004.png,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,8,L2A,2024-05-21 23:27:40,2414216,4,1,browse,"MULTIPOLYGON (((-114.70493 34.57001, -115.4730..."
3,2024-05-21,L2A_MASK,EMIT_L2A_MASK_001_20240521T232740_2414216_004.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,8,L2A,2024-05-21 23:27:40,2414216,4,1,data,"MULTIPOLYGON (((-114.70493 34.57001, -115.4730..."
4,2024-05-21,L2A_RFL,EMIT_L2A_RFL_001_20240521T232740_2414216_004.n...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,8,L2A,2024-05-21 23:27:40,2414216,4,1,unknown,"MULTIPOLYGON (((-114.70493 34.57001, -115.4730..."
5,2024-05-21,L2A_RFLUNCERT,EMIT_L2A_RFLUNCERT_001_20240521T232740_2414216...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,8,L2A,2024-05-21 23:27:40,2414216,4,1,unknown,"MULTIPOLYGON (((-114.70493 34.57001, -115.4730..."


In [5]:
URL = reflectance_granules_df.iloc[0].URL
URL

'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230602T193753_2315313_003/EMIT_L2A_RFL_001_20230602T193753_2315313_003.nc'

In [6]:
filename_base = reflectance_granules_df.iloc[0].filename
filename_base

'EMIT_L2A_RFL_001_20230602T193753_2315313_003.nc'

In [7]:
filename = join("~/data/EMIT_download", filename_base)
filename

'~/data/EMIT_download/EMIT_L2A_RFL_001_20230602T193753_2315313_003.nc'

In [8]:
download_file(URL, filename, capture_output=False, show_command=True)

[2025-01-02 18:50:34 INFO] File already downloaded: ~/data/EMIT_download/EMIT_L2A_RFL_001_20230602T193753_2315313_003.nc


'~/data/EMIT_download/EMIT_L2A_RFL_001_20230602T193753_2315313_003.nc'

In [9]:
directory = "~/data/EMIT_download/scene_directory"
parent_directory = "~/data/EMIT_download"

In [11]:
orbit = 2414216
scene = 2

download_EMIT_L2A_scene(
    orbit=orbit,
    scene=scene,
    parent_directory=parent_directory,
    CMR_results_df=reflectance_granules_df
)

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc
[2025-01-02 18:56:10 INFO] Downloading: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc -> ~/data/EMIT_download/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc


[2025-01-02 18:56:10 INFO] Completed download in 0.02 seconds: ~/data/EMIT_download/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc
[2025-01-02 18:56:10 ERROR] Error on attempt 1: Failed to download URL: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc, attempt 1
Traceback (most recent call last):
  File "/Users/gregoryhalverson/Projects/emit_tools/emit_tools/download_file.py", line 93, in download_file
    raise DownloadFailed(f"Failed to download URL: {URL}, attempt {attempts + 1}")
emit_tools.download_file.DownloadFailed: Failed to download URL: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc, attempt 1
[2025-01-02 18:56:10 WARNING] Waiting for 60 seconds before retrying...
[2025-01-02 18:57:10 INFO]

DownloadFailed: Failed to download file after 3 attempts: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc

In [ ]:
df = reflectance_granules_df.copy()

# Filter the DataFrame to include only rows with the specified orbit number
df = df[df.orbit == orbit]

# Further filter the DataFrame to include only rows with the specified scene number
df = df[df.scene == scene]

df

In [ ]:
str(df[df.filename.apply(lambda filename: filename.endswith(".png"))].iloc[0].filename)